In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [8]:
url = 'https://members.dartconnect.com/history/report/players/match/5e1d25874a6ca334944c8c43'

response = requests.get(url)

In [9]:
page = response.text

In [10]:
soup = BeautifulSoup(page, "lxml")

In [11]:
soup.prettify()

'<!DOCTYPE html>\n<html>\n <head>\n  <title>\n   PP: 8c43\n  </title>\n  <link href="/images/favicon.png" rel="shortcut icon"/>\n  <link href="/css/reports.css" rel="stylesheet"/>\n </head>\n <body>\n  <!-- Google Tag Manager -->\n  <noscript>\n   <iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-5RV476" style="display:none;visibility:hidden" width="0">\n   </iframe>\n  </noscript>\n  <script>\n   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\'//www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);})(window,document,\'script\',\'dataLayer\',\'GTM-5RV476\');\n  </script>\n  <!-- End Google Tag Manager -->\n  <div class="report-container">\n   <div class="report-header">\n    <table border="0" class="freeTable" width="800">\n     <tr>\n      <td align="center" width="140">\n       <a href=

In [99]:
#Find and set the date
datematch = soup.h2.next_element.next_element
daydate = datematch.td.text
day = daydate[0:3]
pdate = daydate[-8:]
if (pdate[0] == " "):
    date = '0' + pdate[1:]
else:
    date = pdate
print(date)

01-13-20


In [36]:
#Identify game detail table
game_pp = soup.find(class_="freeTable playerPref")

In [13]:
#Break table into rows
rows = [row for row in game_pp.find_all("tr")]

In [17]:
#Make function to convert html rows to leg sets
def leggify(x):
    gmst = []
    for i in x:
        if (i.text == '') or (i.text == '\xa0'):
            gmst.append(0)
        else:
            gmst.append(i.get_text())
    return gmst

In [30]:
#Create a set of leg sets
all_legs = []
for row in rows[1:]:
    rowset = row.find_all('td')
    pleg = [leggify(rowset)]
    all_legs.append(pleg)

In [31]:
all_legs

[[['1.1',
   '501 SIDO',
   'Chris Liszkowski',
   0,
   0,
   '44.2',
   '14.7',
   0,
   '2',
   'W',
   '12',
   '6',
   0,
   '501',
   0,
   '34',
   '501',
   0,
   0,
   0,
   0,
   0]],
 [[0,
   'Dustin Scholtes',
   'C',
   0,
   '41.1',
   '13.7',
   0,
   '1',
   0,
   0,
   0,
   0,
   '501',
   '8',
   '36',
   '493',
   0,
   0,
   0,
   0,
   0]],
 [['1.2',
   '501 SIDO',
   'Chris Liszkowski',
   0,
   0,
   '33.0',
   '11.0',
   0,
   '2',
   0,
   0,
   0,
   0,
   '501',
   '303',
   '18',
   '198',
   0,
   0,
   0,
   0,
   0]],
 [[0,
   'Dustin Scholtes',
   'S',
   0,
   '75.2',
   '25.1',
   0,
   '1',
   'W',
   '7',
   '42',
   0,
   '501',
   0,
   '20',
   '501',
   0,
   0,
   0,
   0,
   0]],
 [['1.3',
   '501 SIDO',
   'Chris Liszkowski',
   'C',
   0,
   '48.3',
   '16.1',
   0,
   '1',
   0,
   0,
   0,
   0,
   '501',
   '66',
   '27',
   '435',
   0,
   0,
   0,
   0,
   0]],
 [[0,
   'Dustin Scholtes',
   0,
   0,
   '55.7',
   '18.6',
   0,
   '2',


In [22]:
#Define Player Name function
def PName(matches):
    PName = []
    for leg in matches:
        if leg[0][0] == 0:
            PName.append(leg[0][1])
        else:
            PName.append(leg[0][2])
    return PName

In [23]:
#Define Leg Code function
def LegCode(matches):
    LegCode = []
    for leg in matches:
        if leg[0][0] != 0:
            thisleg = leg[0][0]
            LegCode.append(leg[0][0])
        else:
            LegCode.append(thisleg)
    return LegCode

In [24]:
#Define Game Name function
def Game(matches):
    Game = []
    for leg in matches:
        if leg[0][0] != 0:
            thisgame = leg[0][1]
            Game.append(leg[0][1])
        else:
            Game.append(thisgame)
    return Game

In [25]:
#Define Player 3 Dart Average function
def P3DA(matches):
    P3DA = []
    for leg in matches:
        if leg[0][0] != 0:
            P3DA.append(leg[0][5])
        elif (leg[0][0] == 0) & (leg[0][4] == 0):
            P3DA.append(leg[0][2])
        else:
            P3DA.append(leg[0][4])
    return P3DA

In [26]:
#Define Leg Win Function
def LegWin(matches):
    LegWin = []
    for leg in matches:
        if leg[0][0] != 0:
            LegWin.append(leg[0][9])
        elif (leg[0][0] == 0) & (leg[0][4] == 0):
            LegWin.append(leg[0][6])
        else:
            LegWin.append(leg[0][8])
    return LegWin

In [100]:
gametable = pd.DataFrame(
            {
                "Date": date,
                "Player": PName(all_legs),
                "Leg Code": pd.Series(LegCode(all_legs), dtype="float32"),
                "Game": Game(all_legs),
                "3DA": pd.Series(P3DA(all_legs), dtype="float64"),
                "W/A": LegWin(all_legs)
            })

In [101]:
gametable.head(11)

Date              Player  Leg Code      Game   3DA W/A
0   01-13-20    Chris Liszkowski       1.1  501 SIDO  44.2   W
1   01-13-20     Dustin Scholtes       1.1  501 SIDO  41.1   0
2   01-13-20    Chris Liszkowski       1.2  501 SIDO  33.0   0
3   01-13-20     Dustin Scholtes       1.2  501 SIDO  75.2   W
4   01-13-20    Chris Liszkowski       1.3  501 SIDO  48.3   0
5   01-13-20     Dustin Scholtes       1.3  501 SIDO  55.7   W
6   01-13-20                   0       1.3  501 SIDO   0.0   0
7   01-13-20  Daniel K Siemaszko       2.1  301 DIDO  34.0   0
8   01-13-20     Samuel  Casiano       2.1  301 DIDO  41.3   0
9   01-13-20       Chuck Haisler       2.1  301 DIDO  55.8   W
10  01-13-20         Kevin Lally       2.1  301 DIDO  38.3   A